## Accuracy on test set

In [4]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model
import numpy as np

Using TensorFlow backend.


In [ ]:
model = load_model('fully_trained_resnet.h5', compile=True)

In [2]:
preds

array([1.39804329e-76, 1.39804329e-76, 1.39642638e-76, ...,
       1.39804460e-76, 1.39804329e-76, 1.18295070e-76])

In [106]:
datagen = ImageDataGenerator(rescale=1./255)
samples = datagen.flow_from_directory('/home/jupyter/data/test',
                                      class_mode='categorical',
                                      batch_size=1,
                                      shuffle=False,
                                      target_size=(512,512))
samples.shuffle = False
samples.reset()
nums_labels = samples.classes

Found 25250 images belonging to 101 classes.


In [3]:
test_preds = model.predict_generator(samples, steps=25250)

In [4]:
from sklearn.metrics import accuracy_score

In [5]:
test_preds.argmax(axis=1)-1

array([  0,   0,   8, ..., 100, 100, 100])

In [6]:
labels

array([  0,   0,   0, ..., 100, 100, 100], dtype=int32)

In [7]:
accuracy_score(labels, test_preds.argmax(axis=1)-1)

0.8335841584158415

## Test Time Augmentation

In [31]:
# Prepare data for TTA
datagen = ImageDataGenerator(rescale=1./255)
samples = datagen.flow_from_directory('/home/jupyter/data/test',
                                      class_mode='categorical',
                                      batch_size=1,
                                      shuffle=False,
                                      target_size=(512,512))
samples.shuffle = False
samples.reset()
labels = samples.classes

Found 25250 images belonging to 101 classes.


In [3]:
def test_time_prediction(datagen, model, image, n_examples=7):
    augmenter = datagen.flow(image[0], batch_size=1)
    augmenter.shuffle = False
    augmenter.reset()
    preds = model.predict_generator(augmenter, steps=n_examples, verbose=0)
    all_preds = np.mean(preds, axis=0)
    pred = np.argmax(all_preds, axis=-1)
    return pred, all_preds

In [5]:
from sklearn.metrics import accuracy_score
from keras.metrics import top_k_categorical_accuracy

def test_time_augmentation(model, X_samples, y_labels):
    datagen = ImageDataGenerator(rescale=1./255, 
                                 brightness_range=[0.7,1.3],
                                 rotation_range=30,
                                 horizontal_flip=True,
                                 zoom_range=[0.7,1.3],
                                 fill_mode='nearest')
    examples_per_image = 11
    preds = []
    all_preds = []
    for i in range(len(X_samples)):
        if ((i % 500) == 0):
            print(i, 'Predictions Complete')
        pred, all_preds = test_time_prediction(datagen, model, X_samples[i], examples_per_image)
        preds.append(pred)
    return y_labels, preds, all_preds 

In [6]:
y_labels, preds, all_preds = test_time_augmentation(model, samples, labels)

0 Predictions Complete
500 Predictions Complete
1000 Predictions Complete
1500 Predictions Complete
2000 Predictions Complete
2500 Predictions Complete
3000 Predictions Complete
3500 Predictions Complete
4000 Predictions Complete
4500 Predictions Complete
5000 Predictions Complete
5500 Predictions Complete
6000 Predictions Complete
6500 Predictions Complete
7000 Predictions Complete
7500 Predictions Complete
8000 Predictions Complete
8500 Predictions Complete
9000 Predictions Complete
9500 Predictions Complete
10000 Predictions Complete
10500 Predictions Complete
11000 Predictions Complete
11500 Predictions Complete
12000 Predictions Complete
12500 Predictions Complete
13000 Predictions Complete
13500 Predictions Complete
14000 Predictions Complete
14500 Predictions Complete
15000 Predictions Complete
15500 Predictions Complete
16000 Predictions Complete
16500 Predictions Complete
17000 Predictions Complete
17500 Predictions Complete
18000 Predictions Complete
18500 Predictions Complet

In [7]:
preds[:10]

[1, 1, 1, 1, 1, 1, 22, 1, 1, 1]

In [8]:
y_labels[:10] + 1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [9]:
acc = accuracy_score(y_labels+1, preds)
print(acc)

0.8533465346534653


In [6]:
import numpy as np

In [12]:
preds = np.asarray(preds)
np.savetxt('preds.csv', preds, delimiter=',')

y_labels = np.asarray(y_labels)
np.savetxt('y_labels.csv', y_labels, delimiter=',')

np.savetxt('all_preds.csv', all_preds, delimiter=',')

# Plot Confusion Matrix

In [22]:
preds = np.loadtxt('preds.csv').astype(int)
y_labels = np.loadtxt('y_labels.csv').astype(int)

In [25]:
y_labels += 1

In [74]:
y_labels

array([  1,   1,   1, ..., 101, 101, 101])

In [54]:
import pandas as pd
labels = pd.read_csv('food-101/meta/classes.txt', header=None)
labels = labels[0].tolist() 

In [77]:
len(np.unique(y_labels))

101

In [110]:
(len(np.unique(nums_labels)))

101

In [78]:
len(np.unique(preds))

101

In [79]:
len(labels)

101

In [103]:
len(np.unique(y_labels.astype(int)))

101

In [102]:
preds.astype(int)

array([  1,   1,   1, ..., 101, 101, 101])

In [111]:
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title='Confusion Matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=nums_labels)
    fig, ax = plt.subplots(figsize=(20,20))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]-1),
           yticks=np.arange(cm.shape[0]-1),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(1, cm.shape[0]):
        for j in range(cm.shape[1]-1):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    plt.show()
    return ax, cm

In [ ]:
_, cm = plot_confusion_matrix(y_labels+1, preds, classes=labels)

In [98]:
cm

array([[  0,   0,   0, ...,   0,   0,   0],
       [180,   1,   1, ...,   0,   4,   0],
       [  0, 209,   0, ...,   0,   0,   0],
       ...,
       [  3,   0,   0, ...,   0,   1,   0],
       [  0,   0,   0, ..., 187,   0,   0],
       [  1,   0,   0, ...,   0, 235,   0]])

In [71]:
def most_confused(matrix, classes):
    class_count = 0
    confused = []
    for _class in matrix: 
        print(_class)
        label_count = 0
        for label in _class:
            if class_count == label_count:
                pass
            else:
                print(label)
                if label > 10:
                    confused.append([label, classes[class_count], classes[label_count]])
    return confused

In [72]:
confused = most_confused(cm, labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[180   1   1   0   0   0   3   0  12   2   2   0   5   0   3   0   0   1
   0   2   0   1   0   0   0   0   2   0   0   0   0   3   0   0   0   0
   0   0   0   5   0   0   5   0   0   0   3   0   0   1   1   0   0   0
   0   0   0   3   2   2   0   0   0   0   0   0   0   0   0   0   0   0
   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1
   0   0   0   0   0   1   0   0   2   0   4   0]
[  0 209   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   1
   1   0   1   1   0   0   0   0   0   0   0   0   0   0   0   0   1   0
   1   2   0   0   1   0   1   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   1
   0   0   4   0   0  10   0   0   1   0   0   0   0   0   0   0   0   0
   0   0   0

In [68]:
confused

[]